<a href="https://colab.research.google.com/github/dongjaeseo/study/blob/main/ed5011_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, Input, Normalization
from sklearn.model_selection import train_test_split

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ed5011/B0005/data/001_B0005_charge.csv', header = None)

In [ ]:
print(df)

In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/ed5011/B0005/data/002_B0005_discharge.csv', header = None)

In [ ]:
print(df2)


In [ ]:
df3 = pd.concat([df, df2])

In [ ]:
df3

In [ ]:
from tqdm import tqdm
train_x = []
test_x = []

for i in tqdm(range(1, 335)):
    if i % 2 == 1:
        df1 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0005/data/{i:03}_B0005_charge.csv', header = None)
    elif i % 2 == 0:
        df2 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0005/data/{i:03}_B0005_discharge.csv', header = None)
        df = pd.concat([df1,df2])

        train_x.append(df)
    
for i in tqdm(range(1, 335)):
    if i % 2 == 1:
        df1 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0006/data/{i:03}_B0006_charge.csv', header = None)
    elif i % 2 == 0:
        df2 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0006/data/{i:03}_B0006_discharge.csv', header = None)
        df = pd.concat([df1,df2])

        train_x.append(df)

for i in tqdm(range(1, 265)):
    if i % 2 == 1:
        df1 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0018/data/{i:03}_B0018_charge.csv', header = None)
    elif i % 2 == 0:
        df2 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0018/data/{i:03}_B0018_discharge.csv', header = None)
        df = pd.concat([df1,df2])

        train_x.append(df)

for i in tqdm(range(1, 335)):
    if i % 2 == 1:
        df1 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0007/data/{i:03}_B0007_charge.csv', header = None)
    elif i % 2 == 0:
        df2 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0007/data/{i:03}_B0007_discharge.csv', header = None)
        df = pd.concat([df1,df2])

        test_x.append(df)

In [ ]:
train_x = np.array(train_x)
test_x = np.array(test_x)

train_x.shape

In [ ]:
# Finding the data with maximum length = 4254

max = 0
for data in train_x:
    if len(data) > max:
        max = len(data)

max

In [ ]:
# Pad data

x_train = np.zeros([train_x.shape[0], max, 6])
x_test = np.zeros([test_x.shape[0], max, 6])

for i, dat_x in enumerate(train_x):
    if len(dat_x) <= max:
        num = len(dat_x)
        x_train[i][-num:] = dat_x
    else:
        x_train[i] = dat_x[-max:]

for i, dat_x in enumerate(test_x):
    if len(dat_x) <= max:
        num = len(dat_x)
        x_test[i][-num:] = dat_x
    else:
        x_test[i] = dat_x[-max:]

print(x_train.shape) # (466, 4254, 6)
print(x_test.shape) # (167, 4254, 6)

In [ ]:
# y data
df_y1 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0005/label/label.csv', header = None)
df_y2 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0006/label/label_B0006.csv', header = None)
df_y3 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0018/label/label_B0018.csv', header = None)
df_y4 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0007/label/label_B0007.csv', header = None)

df_y = pd.concat([df_y1, df_y2, df_y3])
y_train = np.array(df_y)
y_test = np.array(df_y4)

print(y_train.shape) # (466, 1)
print(y_test.shape) # (167, 1)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size = 0.8, random_state= 0, shuffle = True)

In [ ]:
model = Sequential()
model.add(LSTM(64, input_shape=(x_train.shape[1:])))
model.add(Dropout(0.2))
model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(1))

In [ ]:
model.summary()

In [ ]:
with tf.device('/device:GPU:0'):  
    optim = tf.keras.optimizers.Adam(lr = 1e-3, decay = 1e-5)

    model.compile(loss = 'mse', optimizer = optim, metrics = [tf.keras.metrics.RootMeanSquaredError()])
    history = model.fit(x_train, y_train, epochs = 150, batch_size = 20, verbose = 1)

In [ ]:
model.evaluate(x_test, y_test, batch_size = 20)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['root_mean_squared_error'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['rmse', 'mse'], loc='upper left')
plt.show()

In [ ]:
y_pred = model.predict(x_test)
print(y_pred[-10:])

print("\n")

y_actual = y_test[-10:]
print(y_actual)

In [ ]:
y_diff = y_pred - y_test
percent = 0
for i in range(len(y_test)):
    percent += y_diff[i]/y_test[i]*100

percent = percent /len(y_test)
percent